In [1]:
from evaluation import evaluate_sql
import pandas as pd


def run_evaluate(model: str, dataset: str):
    print("Evaluating model: ", model, "on dataset: ", dataset)
    result = evaluate_sql(
        solution_file=f"spider_data/{dataset}_gold.sql",
        answer_file=f"inference_data/{model}_{dataset}_inf.txt",
        db_root=(
            "spider_data/database" if dataset == "dev" else "spider_data/test_database"
        ),
    )
    result.details_df.to_csv(f"evaluate_data/{model}_{dataset}_eval.csv", index=False)
    return {
        "model": model,
        "dataset": dataset,
        "total": result.summary_data["total"],
        "correct": result.summary_data["correct"],
        "execution_accuracy": result.summary_data["execution_accuracy"],
    }

In [2]:
eval_list = [
    {"model": "llama32_3b_w_ex", "dataset": "dev"},
    # {"model": "llama32_3b_wo_ex", "dataset": "dev"},
    # {"model": "llama32_3b", "dataset": "test"},
    # {"model": "xiyansql_7b_8_cn_prompt", "dataset": "dev"},
    # {"model": "xiyansql_7b_8_en_prompt", "dataset": "dev"},
    # {"model": "xiyansql_7b_8_en_prompt", "dataset": "test"},
    # {"model": "xiyansql_7b_16_cn_prompt", "dataset": "dev"},
    # {"model": "xiyansql_7b_16_en_prompt", "dataset": "dev"},
    # {"model": "xiyansql_7b_16_en_prompt_vllm", "dataset": "dev"},
    # {"model": "xiyansql_7b_16_en_prompt_vllm", "dataset": "test"},
    # {"model": "xiyansql_7b_16_en_prompt_w_ex", "dataset": "dev"},
    # {"model": "xiyansql_7b_16_en_prompt_w_ex", "dataset": "test"},
    # {"model": "xiyansql_7b_16_en_prompt_m_schema", "dataset": "dev"},
    # {"model": "xiyansql_7b_16_en_w_ex", "dataset": "dev"},
    # {"model": "xiyansql_7b_16_en_wo_ex", "dataset": "dev"},
    # {"model": "xiyansql_7b_16_en_w_ex", "dataset": "test"},
    # {"model": "xiyansql_7b_16_en_m_schema", "dataset": "dev"},
]

data = []
for config in eval_list:
    res = run_evaluate(**config)
    data.append(res)

Evaluating model:  llama32_3b_w_ex on dataset:  dev


In [3]:
df = pd.DataFrame(data)
df

,model,dataset,total,correct,execution_accuracy
0,llama32_3b_w_ex,dev,1034,668,64.6
